In [1]:
import os
import datetime
import numpy as np
import pandas as pd
import pandas_datareader.data as pdr
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [2]:
start = (2000, 1, 1)
start = datetime.datetime(*start) 
# end = datetime.date.today() - datetime.timedelta(1) # 어제 날짜
end = datetime.date.today()

In [3]:
df = pdr.DataReader('005930.KS', 'yahoo', start, end)
df.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2000-01-04,6110.0,5660.0,6000.0,6110.0,74195000.0,4740.119629
2000-01-05,6060.0,5520.0,5800.0,5580.0,74680000.0,4328.947754
2000-01-06,5780.0,5580.0,5750.0,5620.0,54390000.0,4359.979492
2000-01-07,5670.0,5360.0,5560.0,5540.0,40305000.0,4297.916992
2000-01-10,5770.0,5580.0,5600.0,5770.0,46880000.0,4476.349121


In [4]:
df.tail()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2021-04-26,83500.0,82600.0,82900.0,83500.0,15489938.0,83500.0
2021-04-27,83300.0,82500.0,83200.0,82900.0,12941533.0,82900.0
2021-04-28,83200.0,82100.0,83200.0,82100.0,15596759.0,82100.0
2021-04-29,82500.0,81500.0,82400.0,81700.0,20000973.0,81700.0
2021-04-30,82100.0,81500.0,81900.0,81500.0,17244262.0,81500.0


In [5]:
look_back = 20
def create_dataset(dataset, look_back):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back):
        a = dataset[i : (i+look_back)]
        dataX.append(a)
        dataY.append(dataset[i+look_back])
    return np.array(dataX), np.array(dataY)

In [6]:
nparr = df['Close'].values[0:] 
nparr.astype('float32')
print(nparr)

[ 6110.  5580.  5620. ... 82100. 81700. 81500.]


In [7]:
scaler = MinMaxScaler(feature_range=(0, 1))
nparr = nparr.reshape(-1, 1)
nptf = scaler.fit_transform(nparr)

In [8]:
nptf

array([[0.03829161],
       [0.0322873 ],
       [0.03274046],
       ...,
       [0.89917299],
       [0.89464144],
       [0.89237567]])

In [9]:
train = nptf

In [10]:
trainX, trainY = create_dataset(train, look_back)

In [11]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))

In [12]:
trainX.shape

(5336, 1, 20)

In [13]:
x_train, x_valid, y_train, y_valid = train_test_split(trainX, 
                                                      trainY, 
                                                      test_size=0.2)
print('x_train shape :', x_train.shape)
print('x_valid shape :', x_valid.shape)

x_train shape : (4268, 1, 20)
x_valid shape : (1068, 1, 20)


In [14]:
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1)) 
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 4)                 400       
_________________________________________________________________
dense (Dense)                (None, 1)                 5         
Total params: 405
Trainable params: 405
Non-trainable params: 0
_________________________________________________________________


In [15]:
EPOCHS = 20

model.fit(x_train, y_train, 
          epochs=EPOCHS, 
          batch_size=1,
          validation_data=(x_valid, y_valid),
          verbose=1)

Epoch 1/20
4268/4268 [==============================] - 19s 4ms/step - loss: 0.0020 - val_loss: 2.5905e-04
Epoch 2/20
4268/4268 [==============================] - 11s 3ms/step - loss: 2.0873e-04 - val_loss: 1.3834e-04
Epoch 3/20
4268/4268 [==============================] - 12s 3ms/step - loss: 1.6252e-04 - val_loss: 1.6496e-04
Epoch 4/20
4268/4268 [==============================] - 11s 3ms/step - loss: 1.2980e-04 - val_loss: 8.3451e-05
Epoch 5/20
4268/4268 [==============================] - 11s 3ms/step - loss: 1.0282e-04 - val_loss: 9.5939e-05
Epoch 6/20
4268/4268 [==============================] - 11s 3ms/step - loss: 1.1138e-04 - val_loss: 8.3593e-05
Epoch 7/20
4268/4268 [==============================] - 11s 3ms/step - loss: 8.7509e-05 - val_loss: 7.2672e-05
Epoch 8/20
4268/4268 [==============================] - 11s 3ms/step - loss: 9.3352e-05 - val_loss: 8.5154e-05
Epoch 9/20
4268/4268 [==============================] - 11s 3ms/step - loss: 9.1186e-05 - val_loss: 1.2655e-04
Epoch

In [16]:
lastX = trainX[-1]
lastX = np.reshape(lastX, (1, 1, look_back))
lastY = model.predict(lastX)
lastY = scaler.inverse_transform(lastY)
print("오늘 예측 값은 :", lastY)

오늘 예측 값은 : [[81722.586]]


In [17]:
print("오늘 값은")
nparr[-1]

오늘 값은


array([81500.])

In [18]:
lastX = nptf[-look_back:]
lastX = np.reshape(lastX, (1, 1, look_back))
lastY = model.predict(lastX)
lastY = scaler.inverse_transform(lastY)
print("내일 예측 값은 :", lastY)

내일 예측 값은 : [[81458.195]]
